In [ ]:

# Amount (input) -> { last ah check pannanum balance ah vida exceed aagaama irukkanu }

# card no (input) -> card no laam sariyaa irukkaanu

# cvv (input) -> cvv antha card kkanathaanu naama pakkanum

# security pin (input) -> security pin um antha card kkanathaanu naama pakkanum

# thiruppi theva pattra matter ah oru variable la input vaangum pothae store aaganum update-um aaganum

# Ellaam match aagittaa ippa check pannanum {} & ok naa -> balance ah update panni vidranum, illainaa insufficient msge kodukkanum
---------
# card details venum -> cross check um nadakkanum

# cross check epdinaa -> condition based details ah variable ah update panni cross verify

# update



In [1]:
import psycopg2
import pandas as pd

In [2]:
conn = psycopg2.connect(host = 'localhost',password = 'PASSWORD', port = 5432, user = 'postgres', database = 'Bank_ATM')
conn.autocommit = True
cur = conn.cursor()

In [3]:
# SBI ACCOUNT'S TABLE
cur.execute("select * from sbi")
Acc = cur.fetchall()
Accounts = pd.DataFrame(Acc)
Ac_display=Accounts.loc[:,:5]
Ac_display.columns = ['Ac/No','Name','Address Code','Balance','Phone','PIN']
print(Ac_display)

         Ac/No    Name Address Code  Balance       Phone   PIN
0  33171516990   Yusuf          rmd    11750  6381810099  3215
1  32435343353   Saran          slm    34810  8972389663  2354
2  33187183334  Madhan          npt    34001  6389289233  4393


In [4]:
# DEBIT CARD DETAILS
cur.execute("select * from debit_card")
DC = cur.fetchall()
Debit_Card = pd.DataFrame(DC)
Debit_Card.columns = ["Card_NO",'Ac/No','Name',"CVV","PIN"]
print(Debit_Card)

    Card_NO        Ac/No    Name  CVV   PIN
0  12345688  33187183334  Madhan  245  4393
1  12345678  33171516990   Yusuf  324  3215
2  12345698  32435343353   Saran  372  2354


In [6]:
class ATM:
    def Amount(self):
        self.Name = input('Name :' )
        Name = self.Name
        self.Amount = int(input('Amount :'))
        cur.execute("select balance from sbi where name =%s;", [Name])
        Amount_fetch = cur.fetchall()
        while True:
            for i in Amount_fetch:pass
            if self.Amount > i[0] :
                    print("insufficient Balance!")
                    self.Amount = int(input('Amount :'))
            else:
                break
                    
    #..........................................................................................................#
    
    def card_details(self):
        #..................................Card_No.........................................#   
        
        Name = self.Name
        Card_NO = int(input("card no :"))
        cur.execute("select card_no from debit_card where name = %s",[Name])
        card = cur.fetchall()
        while True:
            for i in card:pass
            if Card_NO != i[0]:
                print("Enter correct card_no")
                Card_NO = int(input("card no :"))
            else:
                break
                
        #..................................CVV.........................................#   
        
        CVV = int(input("CVV :"))
        cur.execute("select cvv from debit_card where name = %s",[Name])
        cvv = cur.fetchall()
        while True:
            for i in cvv:pass
            if CVV != i[0]:
                print("Enter correct cvv")
                CVV = int(input("cvv :"))
            else:
                break
                
        #..................................PIN.........................................#   
        
        PIN = int(input("PIN :"))
        cur.execute("select security_pin from debit_card where name = %s",[Name])
        pin = cur.fetchall()
        while True:
            for i in pin:pass
            if PIN != i[0]:
                print("Enter correct PIN")
                PIN = int(input("PIN :"))
            else:
                    break

    
    def balance_update(self):
        Name = self.Name
        Amount = self.Amount
        cur.execute("update sbi set balance = (balance - %s) where name = %s", [Amount,Name])
        cur.execute("select * from audit order by timestamp desc")
        audit = cur.fetchone()
        df = pd.DataFrame(audit)
        DF = df.transpose()
        DF.columns = ['Transaction_id','Ac_No','Name','Withdrawn Amount','Timestamp']
        print("! SUCCESSFULLY WITHDRAWN !")
        return DF
    
    

In [7]:
ATM = ATM ()

In [8]:
ATM.Amount()
ATM.card_details()
ATM.balance_update()

Name :Saran
Amount :130
card no :12345698
CVV :372
PIN :2354
! SUCCESSFULLY WITHDRAWN !


,Transaction_id,Ac_No,Name,Withdrawn Amount,Timestamp
0,8b91d032,32435343353,Saran,130,2023-02-21 10:13:39


In [9]:
# Audit Log
cur.execute("select * from audit")
ad = cur.fetchall()
audit = pd.DataFrame(ad)
audit.columns = ["Transaction_id",'Ac/No','Name',"Withdrawn Amount","Timestamp"]
print(audit)

  Transaction_id        Ac/No    Name  Withdrawn Amount            Timestamp
0       41fd3cd6  33171516990   Yusuf               100  2023-02-20 10:45:36
1       3500f61b  33171516990   Yusuf               150  2023-02-20 10:47:54
2       6124643d  32435343353   Saran               190  2023-02-20 10:49:35
3       7229ce93  33187183334  Madhan               999  2023-02-20 10:50:57
4       8b91d032  32435343353   Saran               130  2023-02-21 10:13:39
